# Python实例练习：用户行为模式分析

## 项目背景

### 1.数据来源

* 本次项目基于天池公开数据，通过对相关指标进行用户行为分析
* 数据文件下载地址：https://tianchi.aliyun.com/dataset/dataDetail?dataId=20317

### 2.分析目的

* 1.计算日PV、日UV
* 2.付费率情况
* 3.复购率情况
* 4.构建漏斗模型

### 3.数据简介

* user_id：用户ID，数据已脱敏
* item_id：商品ID，数据已脱敏
* behavior_type：行为类型,共有4类，包括点击、收藏、加购物车、支付等4种行为，分别用数字1、2、3、4表示）
* user_geohash：地理位置
* item_category：品类ID,即商品所属品类
* time：发生时间

## 1. 导入相关数据库

In [1]:
import numpy as np
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import cufflinks
from plotly.offline import iplot,init_notebook_mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

## 2.读取数据并理解

In [2]:
# 无参数导入csv文件发现user_id等4个字段的数据类型为int，因此，导入时直接全部设为string。
data=pd.read_csv('tianchi_fresh_comp_train_user.csv',encoding="utf-8",dtype=str)

In [3]:
data.head(5)

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,10001082,285259775,1,97lk14c,4076,2014-12-08 18
1,10001082,4368907,1,NaN,5503,2014-12-12 12
2,10001082,4368907,1,NaN,5503,2014-12-12 12
3,10001082,53616768,1,NaN,9762,2014-12-02 15
4,10001082,151466952,1,NaN,5232,2014-12-12 11


In [4]:
print(data.shape)
print(data.info())

(23291027, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23291027 entries, 0 to 23291026
Data columns (total 6 columns):
user_id          object
item_id          object
behavior_type    object
user_geohash     object
item_category    object
time             object
dtypes: object(6)
memory usage: 1.0+ GB
None


## 3.数据清洗

## 3.1.缺失值处理

In [5]:
#统计缺失率
data.apply(lambda x:np.sum(x.isnull())/len(x),axis=0)

user_id          0.000000
item_id          0.000000
behavior_type    0.000000
user_geohash     0.683139
item_category    0.000000
time             0.000000
dtype: float64

可以看出：user_geohash的字段存在缺失值，且缺失率高达68.3%。因本次分析目的不涉及地理位置，为提高数据处理效率，删除该字段。

In [6]:
data.drop('user_geohash',axis=1,inplace=True)

## 3.2.重复值处理

In [7]:
# 删除整行相同的重复数据
data.drop_duplicates(inplace=True)

## 3.3.增加新变量

In [ ]:
# 时间字段的处理
data['time']=pd.to_datetime(data['time'],errors='coerce')
data['time'].head()
data['date']=pd.to_datetime(data['time'].dt.date,errors='coerce')
data['hour']=data['time'].dt.hour.astype('int')

## 3.4.异常值处理

In [ ]:
np.set_printoptions(suppress= True)
pd.set_option('display.float_format',lambda x: '%.2f'% x)#设置数据的显示为浮点型。
data.describe(include='all')

* 数据中除了时间变量外，其余5个变量均为字符串字段，可以看出数据无异常。

# 4.数据分析

## 4.1.pv和uv分析
* pv：页面访问量，页面被刷新一次就计算一次
* uv：访客访问量，访问网站的一个客户端为一个访客

### 1.日期变化特征

In [ ]:
# 计算日pv
pv_daily=data.groupby('date').count()[['user_id']].rename(columns={'user_id':'pv'})

In [ ]:
# 计算日uv:排除用户一天多次登录的情况
uv_daily=data.groupby('date')[['user_id']].apply(lambda x: x.drop_duplicates().count())\
.rename(columns={'user_id':'uv'})
# 去重值计算也可以写成data.groupby('date').nunique()[['user_id']]

In [ ]:
# 将日pv和日uv合并在一起
pv_uv_daily=pd.merge(left=pv_daily,right=uv_daily,how='outer',left_index=True ,right_index=True)

In [ ]:
# 计算两者的相关系数
corr_pv_uv_daily=pv_uv_daily.corr(method='pearson')
print('日pv和日uv的相关系数为：%.2f'% corr_pv_uv_daily.iloc[0,1])

In [ ]:
# 计算独立访客日平均浏览量
pv_uv_daily['ratio']=pv_uv_daily['pv']/pv_uv_daily['uv']
print(pv_uv_daily[['ratio']].describe())
pv_uv_daily['ratio'].iplot(mode='lines+markers',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Date',
    yTitle='pv/uv',
    title='pv/uv over time')

In [ ]:
# 日pv和日uv数据可视化
pv_uv_daily[['pv','uv']].iplot(mode='lines+markers',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Date',
    yTitle='pv',
    secondary_y = 'uv',
    secondary_y_title='uv',
    title='pv and uv over time')

从以上分析可以看出：
* 1、日pv和日uv关系密切，相关系数为0.93，升降趋势相同。
* 2、双十二期间，日pv和日uv均达到峰值，当天用户最为活跃，平均一个独立访客打开页面44次。

### 2.时间变化特征

In [ ]:
# 计算小时pv
pv_hour=data.groupby('hour').count()[['user_id']].rename(columns={'user_id':'pv'})

In [ ]:
# 计算小时uv
uv_hour=data.groupby('hour').nunique()[['user_id']].rename(columns={'user_id':'uv'})

In [ ]:
# 将小时pv和小时uv合并在一起
pv_uv_hour=pd.merge(left=pv_hour,right=uv_hour,how='outer',left_index=True ,right_index=True)

In [ ]:
# 计算两者的相关系数
corr_pv_uv_hour=pv_uv_hour.corr(method='pearson')
print('小时pv和小时uv的相关系数为：%.2f'% corr_pv_uv_hour.iloc[0,1])

In [ ]:
# 计算独立访客小时平均浏览量
pv_uv_hour['ratio']=pv_uv_hour['pv']/pv_uv_hour['uv']
print(pv_uv_hour[['ratio']].describe())
pv_uv_hour['ratio'].iplot(mode='lines+markers',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Date',
    yTitle='pv/uv',
    title='pv/uv over time')

In [ ]:
# 小时pv和小时uv数据可视化
pv_uv_hour[['pv','uv']].iplot(mode='lines+markers',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Date',
    yTitle='pv',
    secondary_y = 'uv',
    secondary_y_title='uv',
    title='pv and uv over time')

从以上分析可以看出：
* 0点-5点期间，pv和uv同步下降，用户活跃度降低。
* 5点-10点期间，uv急剧上升，大部分用户已经激活，但用户行为不活跃，访问量较低。
* 晚上18点之后，pv急剧上升，并在22点达到峰值，说明晚上是用户访问的活跃时间段。

### 3.不同行为类型的pv分析

In [ ]:
pv_detail=pd.pivot_table(data=data,
    values='user_id',
    index='hour',
    columns='behavior_type',
    aggfunc=np.size)

In [ ]:
pv_detail[['1', '2', '3','4']].iplot(
    mode="lines",
    opacity=0.8,
    size=8,
    symbol=1,
    secondary_y = ['2','3','4'],
    xTitle='hour',
    title='behavior_type and pv over Time')

从以上分析可以看出：
* 1、四种用户行为的波动情况和pv的整体波动情况基本一致。
* 2、点击（1）这一行为的pv访问量远高于其他三种用户行为，之后依次是加购物车（3）、收藏（2）、支付（4）。

## 4.2.用户消费行为分析

### 1.用户购买次数分析

In [ ]:
data_buy=data[data['behavior_type']=='4'].groupby('user_id').size()
print(data_buy.describe())
print(data_buy.quantile([0.7,0.8,0.9]))

In [ ]:
sum(data_buy[data_buy.values>10])/sum(data_buy)

In [ ]:
data_buy.iplot(
    kind='hist',
    bins=100,
    xTitle='purchase times',
    linecolor='black',
    histnorm='percent',
    yTitle='percentage (%)',
    title='User purchase times')

从以上分析可知：
* 1、一半的用户消费次数低于8次，80%的用户消费次数低于17次,但位于金字塔顶端20%的用户消费次数最多可达331次，应对该部分高粘性用户实行特定的粘性政策。
* 2、消费次数高于10次的用户贡献了3/4的总消费数，应重点关注该部分用户的维护。

### 2.分析ARPPU
* ARPPU(average revenue per paying user):即从每个付费用户身上得到的收入。
* ARPPU=total revenue/user number
* 因为表中没有用户消费金额，因此采用消费次数代替消费金额。

In [ ]:
# 统计每个用户每天的消费次数
data_buy_user=data[data['behavior_type']=='4'].groupby(['date','user_id'])['behavior_type'].count().reset_index().rename(columns={'behavior_type':'消费次数'})
data_buy_user

In [ ]:
# 日人均消费次数=日销售总次数/日消费人数
data_buy_per=pd.DataFrame(data_buy_user.groupby('date').sum()['消费次数']/\
                          data_buy_user.groupby('date').count()['消费次数']).rename(columns={'消费次数':'日均消费次数'})
data_buy_per.insert(0,'日销售总次数',data_buy_user.groupby('date').sum()['消费次数'])
data_buy_per.insert(1,'日消费人数',data_buy_user.groupby('date').count()['消费次数'])

In [ ]:
data_buy_per.describe()

In [ ]:
data_buy_per['日均消费次数'].iplot(mode='lines')

从以上分析可以得出：
* 用户每日消费次数在1-2之间波动，但在双十二期间达到峰值。

### 3.复购情况分析
* 复购情况：即两天以上有购买行为，一天多次购买算一次。
* 复购率=有复购行为的用户数/有购买行为的用户总数

In [ ]:
data_rebuy=data[data['behavior_type']=='4'].groupby('user_id')[['date']].apply(lambda x: x.nunique()).\
            rename(columns={'date':'rebuy_count'})

In [ ]:
# 计算复购率
print('一个月内复购率为：',round(data_rebuy[data_rebuy['rebuy_count']>=2].count()\
                        /data_rebuy.count(),2).values[0])

In [ ]:
# 看一下复购率情况
data_rebuy.iplot(kind='hist',bins=100)

* 这里的分析结果其实和4.2.1.用户购买次数分析中一致，不再赘述

In [ ]:
# 所有复购时间间隔消费次数分析
data_buy_user3=data[data.behavior_type=='4'].groupby(['user_id','date']).behavior_type.count().reset_index()

In [ ]:
data_buy_user3.head()

In [ ]:
# 使用差分函数diff计算每个用户每次的时间间隔
data_buy_user3=data_buy_user3.groupby('user_id')['date'].apply\
              (lambda x: x.sort_values().diff(1).dropna()).map(lambda x: x.days)
data_buy_user3.describe()

In [ ]:
# 数据可视化
data_buy_user3.value_counts().iplot(kind='bar')

从以上分析可知：
* 大多数付费用户在1-5天之内复购，相隔10天以上复购的用户很少，说明用户对APP的忠诚度和需求度较高。
* 用户对APP的复购需求大，因此应重视在10天之内激活用户复购。

In [ ]:
import seaborn as sns
print(data_buy_user3.reset_index().groupby('user_id').date.mean().describe())
sns.distplot(data_buy_user3.reset_index().groupby('user_id').date.mean())


* 不同用户平均复购时间接近正态分布，以4作为峰值，向两边呈下降趋势。
* 大部分用户平均复购时间集中在1-6天内。

### 4.漏斗流失分析

In [ ]:
data_processing=data['behavior_type'].value_counts()
data_all=data['behavior_type'].count()

In [ ]:
data.groupby('behavior_type').size()

In [ ]:
# 计算各个环节的流失率（收藏不是必要的用户行为，故不纳入分析）
print('浏览-点击：流失率等于%.2f%%'%(((data_all-data_processing[0])*100)/data_all))
print('点击-加入购物车：流失率等于%.2f%%'%(((data_processing[0]-data_processing[2])*100)/data_processing[0]))
print('加入购物车-支付：流失率等于%.2f%%'%(((data_processing[2]-data_processing[3])*100)/data_processing[2]))

从以上分析可以得知：
* 点击-加入购物车的流失率非常高，说明用户在浏览时被封面吸引进行点击，但商品的具体介绍部分无法进一步吸引顾客。
* 因此，需要重视该环节的优化，做好商品的具体展示。